In [4]:
#!pip install PyYAML

In [49]:
import cv2
import numpy as np
import os
import yaml
from yaml.loader import SafeLoader

In [50]:
# load YAML
with open('data.yaml',mode='r') as f:
    data_yaml = yaml.load(f,Loader=SafeLoader)
    
labels = data_yaml['names']
print(labels)

['menu', 'menu-bar', 'menu-icon', 'decorators']


In [51]:
# load YOLO model
yolo = cv2.dnn.readNetFromONNX('/home/lenovo/temp-image-detec/Model5/weights/best.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [65]:
# load the image
img = cv2.imread('/home/lenovo/temp-image-detec/image 244.png')
image = img.copy()
row, col, d = image.shape


# get the YOLO prediction from the the image
# step-1 convert image into square image (array)
max_rc = max(row,col)
input_image = np.zeros((max_rc,max_rc,3),dtype=np.uint8)
input_image[0:row,0:col] = image
# step-2: get prediction from square array
INPUT_WH_YOLO = 640
blob = cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WH_YOLO,INPUT_WH_YOLO),swapRB=True,crop=False)
yolo.setInput(blob)
preds = yolo.forward() # detection or prediction from YOLO

In [66]:
print(preds.shape)

(1, 25200, 9)


In [76]:
# Non Maximum Supression
# step-1: filter detection based on confidence (0.4) and probability score (0.25)
detections = preds[0]
boxes = []
confidences = []
classes = []

# widht and height of the image (input_image)
image_w, image_h = input_image.shape[:2]
x_factor = image_w/INPUT_WH_YOLO
y_factor = image_h/INPUT_WH_YOLO

for i in range(len(detections)):
    row = detections[i]
    confidence = row[4] # confidence of detection an object
    if confidence > 0.055:
        class_score = row[5:].max() # maximum probability from 20 objects
        class_id = row[5:].argmax() # get the index position at which max probabilty occur
        
        if class_score > 0.055:
            cx, cy, w, h = row[0:4]
            # construct bounding from four values
            # left, top, width and height
            left = int((cx - 0.5*w)*x_factor)
            top = int((cy - 0.5*h)*y_factor)
            width = int(w*x_factor)
            height = int(h*y_factor)
            
            box = np.array([left,top,width,height])
            
            # append values into the list
            confidences.append(confidence)
            boxes.append(box)
            classes.append(class_id)
            
# clean
boxes_np = np.array(boxes).tolist()
confidences_np = np.array(confidences).tolist()

# NMS
index = cv2.dnn.NMSBoxes(boxes_np,confidences_np,0.25,0.45).flatten()


In [77]:
confidences_np


[0.06112714111804962,
 0.06750021874904633,
 0.061311498284339905,
 0.0580921471118927,
 0.06524455547332764,
 0.05946242809295654,
 0.07778619974851608,
 0.08085272461175919,
 0.05601772293448448,
 0.06564705818891525,
 0.06762740015983582,
 0.06538458168506622,
 0.0616866759955883,
 0.06479623168706894,
 0.06602216511964798,
 0.07151811569929123,
 0.07930142432451248,
 0.10626809298992157,
 0.0994148701429367,
 0.055450841784477234,
 0.05940759927034378,
 0.07341519743204117,
 0.06659888476133347,
 0.07250063121318817,
 0.07617131620645523,
 0.0649048462510109,
 0.05545021966099739,
 0.06335709244012833,
 0.06668435782194138,
 0.0695316269993782,
 0.06839379668235779,
 0.07108091562986374,
 0.09841851145029068,
 0.0982341468334198,
 0.062420785427093506,
 0.0663292333483696,
 0.0761517658829689,
 0.07903257757425308,
 0.07812485098838806,
 0.07445693761110306,
 0.07666155695915222,
 0.07791288197040558,
 0.08699978142976761,
 0.0984436571598053,
 0.12719053030014038,
 0.1161080375313

In [78]:
# Draw the Bounding
for ind in index:
    # extract bounding box
    x,y,w,h = boxes_np[ind]
    bb_conf = int(confidences_np[ind]*100)
    classes_id = classes[ind]
    class_name = labels[classes_id]
    
    text = f'{class_name}: {bb_conf}%'
    
    cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.rectangle(image,(x,y-30),(x+w,y),(255,255,255),-1)
    
    cv2.putText(image,text,(x,y-10),cv2.FONT_HERSHEY_PLAIN,0.7,(0,0,0),1)
    


In [79]:
cv2.imshow('original',img)
cv2.imshow('yolo_prediction',image)
cv2.waitKey(0)
cv2.destroyAllWindows()